In [1]:
# 0. 최초 데이터 경로
money_data = '/content/Dataset/'

In [16]:
# 1.폴더 구조 만들기
!mkdir -p /content/Dataset/images
!mkdir -p /content/Dataset/images/train
!mkdir -p /content/Dataset/images/val
!mkdir -p /content/Dataset/labels
!mkdir -p /content/Dataset/labels/train
!mkdir -p /content/Dataset/labels/val

In [17]:
# 2. Dataset split
import glob, shutil, random, os

won_list = ['10', '50', '100', '500', '1000', '5000', '10000', '50000']
data_path = '/content/Dataset/'

# Training과 Validation 비율 설정 (예: 8:2)
split_ratio = 0.8

# 클래스 별로 데이터 분할 및 이동
for won in won_list:
    class_dir = os.path.join(data_path, won)

     # 클래스 폴더 내의 이미지와 JSON 파일 목록 가져오기
    image_files = glob.glob(os.path.join(class_dir, f'{won}_*.jpg'))
    json_files = glob.glob(os.path.join(class_dir, f'{won}_*.json'))

    # 데이터를 무작위로 섞기
    random.shuffle(image_files)
    random.shuffle(json_files)

    # 데이터 분할 인덱스 계산
    split_index = int(len(image_files) * split_ratio)

    # Training set과 Validation set으로 분할
    train_image_files = image_files[:split_index]
    validation_image_files = image_files[split_index:]

    train_json_files = [os.path.join(class_dir, os.path.basename(image_file).replace('.jpg', '.json')) for image_file in train_image_files]
    validation_json_files = [os.path.join(class_dir, os.path.basename(image_file).replace('.jpg', '.json')) for image_file in validation_image_files]

    # 이미지 파일과 JSON 파일을 해당 디렉토리로 이동
    for image_file, json_file in zip(train_image_files, train_json_files):
        # 수정: 경로를 주어진 경로로 변경
        shutil.move(image_file, os.path.join(data_path, 'images', 'train', os.path.basename(image_file)))
        shutil.move(json_file, os.path.join(data_path, 'labels', 'train', os.path.basename(json_file)))

    for image_file, json_file in zip(validation_image_files, validation_json_files):
        # 수정: 경로를 주어진 경로로 변경
        shutil.move(image_file, os.path.join(data_path, 'images', 'val', os.path.basename(image_file)))
        shutil.move(json_file, os.path.join(data_path, 'labels', 'val', os.path.basename(json_file)))

print("데이터 분할이 완료되었습니다.")

데이터 분할이 완료되었습니다.


In [18]:
# 3. 데이터 분할이 잘 되었는지 확인
data_path = '/content/Dataset/labels/'

# 데이터 분할 정보
temp_list = ['train', 'val']

# 이미지와 JSON 파일 수 세기
for temp in temp_list:
    image_dir = os.path.join(data_path,  temp)
    json_dir = os.path.join(data_path, temp)

    # 이미지 파일 수 세기
    image_files = glob.glob(os.path.join(image_dir, '*.jpg'))
    num_images = len(image_files)

    # JSON 파일 수 세기
    json_files = glob.glob(os.path.join(json_dir, '*.json'))
    num_json = len(json_files)

    print(f"Data Split: {temp}, Currency Class: {won}")
    print(f"Number of Images: {num_images}")
    print(f"Number of JSON Files: {num_json}")
    print("-" * 40)

Data Split: train, Currency Class: 50000
Number of Images: 0
Number of JSON Files: 0
----------------------------------------
Data Split: val, Currency Class: 50000
Number of Images: 0
Number of JSON Files: 0
----------------------------------------


In [7]:
# train 파일 정보 추출
import json

# JSON 파일이 있는 디렉토리 경로 설정
json_path = '/content/Dataset/labels/train'

# 클래스 정보 매핑 (앞뒷면 구분 없애기)
class_mapping = {
    'Ten_back': 0,
    'Ten_front': 0,
    'Fifty_back': 1,
    'Fifty_front': 1,
    'Hundred_back': 2,
    'Hundred_front': 2,
    'Five_Hundred_back': 3,
    'Five_Hundred_front': 3,
    'Thousand_back': 4,
    'Thousand_front': 4,
    'Five_Thousand_back': 5,
    'Five_Thousand_front': 5,
    'Ten_Thousand_back': 6,
    'Ten_Thousand_front': 6,
    'Fifty_Thousand_back': 7,
    'Fifty_Thousand_front': 7
}

# JSON 파일 처리
json_files = [f for f in os.listdir(json_path) if f.endswith('.json')]

for json_file_name in json_files:
    # JSON 파일 불러오기
    with open(os.path.join(json_path, json_file_name), 'r') as json_file:
        data = json.load(json_file)

    # 도형 정보 중 하나를 선택하여 클래스 이름 추출
    class_id = class_mapping[data['shapes'][0]['label']]

    # 위치 정보 추출
    x1, y1 = data['shapes'][0]['points'][0]
    x2, y2 = data['shapes'][0]['points'][1]
    image_width = data['imageWidth']
    image_height = data['imageHeight']

    # 이미지 축소 비율 적용
    x_center = (x1 + x2) / 2 / image_width
    y_center = (y1 + y2) / 2 / image_height
    width = (x2 - x1) / image_width
    height = (y2 - y1) / image_height

    # YOLO 형식의 레이블 생성
    yolo_label = f"{class_id} {x_center} {y_center} {width} {height}"

    # YOLO 레이블을 텍스트 파일로 저장
    output_txt_path = os.path.join(json_path, os.path.splitext(json_file_name)[0] + '.txt')
    with open(output_txt_path, 'w') as output_file:
        output_file.write(yolo_label)

print("YOLO 형식의 label로 변환 및 저장이 완료되었습니다.")

YOLO 형식의 label로 변환 및 저장이 완료되었습니다.


In [8]:
# val 파일 정보 추출
import json

# JSON 파일이 있는 디렉토리 경로 설정
json_path = '/content/Dataset/labels/val'

# 클래스 정보 매핑 (앞뒷면 구분 없애기)
class_mapping = {
    'Ten_back': 0,
    'Ten_front': 0,
    'Fifty_back': 1,
    'Fifty_front': 1,
    'Hundred_back': 2,
    'Hundred_front': 2,
    'Five_Hundred_back': 3,
    'Five_Hundred_front': 3,
    'Thousand_back': 4,
    'Thousand_front': 4,
    'Five_Thousand_back': 5,
    'Five_Thousand_front': 5,
    'Ten_Thousand_back': 6,
    'Ten_Thousand_front': 6,
    'Fifty_Thousand_back': 7,
    'Fifty_Thousand_front': 7
}

# JSON 파일 처리
json_files = [f for f in os.listdir(json_path) if f.endswith('.json')]

for json_file_name in json_files:
    # JSON 파일 불러오기
    with open(os.path.join(json_path, json_file_name), 'r') as json_file:
        data = json.load(json_file)

    # 도형 정보 중 하나를 선택하여 클래스 이름 추출
    class_id = class_mapping[data['shapes'][0]['label']]

    # 위치 정보 추출
    x1, y1 = data['shapes'][0]['points'][0]
    x2, y2 = data['shapes'][0]['points'][1]
    image_width = data['imageWidth']
    image_height = data['imageHeight']

    # 이미지 축소 비율 적용
    x_center = (x1 + x2) / 2 / image_width
    y_center = (y1 + y2) / 2 / image_height
    width = (x2 - x1) / image_width
    height = (y2 - y1) / image_height

    # YOLO 형식의 레이블 생성
    yolo_label = f"{class_id} {x_center} {y_center} {width} {height}"

    # YOLO 레이블을 텍스트 파일로 저장
    output_txt_path = os.path.join(json_path, os.path.splitext(json_file_name)[0] + '.txt')
    with open(output_txt_path, 'w') as output_file:
        output_file.write(yolo_label)

print("YOLO 형식의 label로 변환 및 저장이 완료되었습니다.")

YOLO 형식의 label로 변환 및 저장이 완료되었습니다.


In [9]:
# 변환 잘 됐는지 갯수 확인
directory_path1 = '/content/Dataset/labels/train'  # 확인하려는 디렉토리 경로로 변경하세요.
# 디렉토리 내 파일 리스트 얻기
files1 = os.listdir(directory_path1)
# 파일의 갯수 출력
file_count1 = len(files1)
print(f'디렉토리 {directory_path1}에 있는 파일 갯수: {file_count1}')

directory_path2 = '/content/Dataset/labels/val'  # 확인하려는 디렉토리 경로로 변경하세요.
# 디렉토리 내 파일 리스트 얻기
files2 = os.listdir(directory_path2)
# 파일의 갯수 출력
file_count2 = len(files2)
print(f'디렉토리 {directory_path2}에 있는 파일 갯수: {file_count2}')

디렉토리 /content/Dataset/labels/train에 있는 파일 갯수: 0
디렉토리 /content/Dataset/labels/val에 있는 파일 갯수: 0


In [10]:
import yaml

# 클래스 레이블과 클래스 개수를 정의
class_names = {
    0: '10',
    1: '50',
    2: '100',
    3: '500',
    4: '1000',
    5: '5000',
    6: '10000',
    7: '50000'
}

num_classes = len(class_names)

# 데이터셋 정보를 정의합니다.
dataset_info = {
    'names': class_names,   # 클래스 레이블 이름
    'nc': num_classes,      # 클래스 개수
    'train': '/content/Dataset/images/train',  # 학습 데이터셋 경로
    'val': '/content/Dataset/images/val'       # 검증 데이터셋 경로
}

# YAML 파일로 데이터셋 정보를 저장합니다.
with open('dataset_info.yaml', 'w') as yaml_file:
    yaml.dump(dataset_info, yaml_file)

# 생성 완료 메시지를 출력합니다.
print("데이터셋 정보가 담긴 YAML 파일이 생성되었습니다.")

데이터셋 정보가 담긴 YAML 파일이 생성되었습니다.


In [12]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.9/802.9 kB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [14]:
from ultralytics import YOLO
model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

from ultralytics import YOLO
model = YOLO('yolov8n.yaml').load('yolov8n.pt')

# Use the model
model.train(data="dataset_info.yaml",
            epochs=100,
            optimizer='auto',
            degrees=180,    # image rotation (+/- deg)
            flipud=0.5,     # image flip up-down (probability)
            fliplr=0.5,     # image flip left-right (probability)
            translate=0.1,	# image translation (+/- fraction)
            patience=4,
            mixup=0.15      # image mixup (probability)
            )  # train the model

Transferred 355/355 items from pretrained weights
Transferred 355/355 items from pretrained weights
Ultralytics YOLOv8.2.60 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=dataset_info.yaml, epochs=100, time=None, patience=4, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False,

FileNotFoundError: [34m[1mtrain: [0mError loading data from /content/Dataset/images/train
See https://docs.ultralytics.com/datasets for dataset formatting guidance.

In [ ]:
model.predict(source='https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/Currency_South_Korea.jpg/1280px-Currency_South_Korea.jpg')